In [ ]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from PIL import Image

# Path to the directory where the trained CNN model is saved
model_dir = #Needs to be defined
model_file = 'best_model.keras' 

model = load_model(os.path.join(model_dir, model_file))

# Directory containing the subfolders with images
# Requires 1 subfolder with images - name is irrelevant
main_image_dir = #Needs to be defined

# Define datagen
gen = ImageDataGenerator(
    rescale=1/255.,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=0.4, 
    zoom_range=0.4
)

# Prepare DataFrame to store results
class_percentages_list = []

# Iterate over each subfolder in the main image directory
for folder_name in os.listdir(main_image_dir):
    folder_path = os.path.join(main_image_dir, folder_name)
    print("Processing folder:", folder_path)

        # Create a generator to read images found in the directory
    images = gen.flow_from_directory(
            folder_path,
            target_size=(64, 64),  # Match the input size of your model
            color_mode='grayscale',  # Adjust if your model was trained on color images
            batch_size=1,
            class_mode=None,  
            shuffle=False
        )

    predictions = model.predict(images, steps=len(images))
    predicted_classes = np.argmax(predictions, axis=1)

    # Calculate class percentages
    unique, counts = np.unique(predicted_classes, return_counts=True)
    percentages = dict(zip(unique, counts * 100 / len(predicted_classes)))
    percentages['folder_name'] = folder_name
    class_percentages_list.append(percentages)
    print(f"Class percentages for {folder_name}: {percentages}")
    

class_percentages_df = pd.DataFrame(class_percentages_list)

# Set folder_name as the first column
cols = ['folder_name'] + [col for col in class_percentages_df if col != 'folder_name']
class_percentages_df = class_percentages_df[cols]

# Save class percentages to a single CSV file
output_file = #Give directions here
class_percentages_df.to_csv(output_file, index=False)
print("Results saved to", output_file)
